In [18]:
# prompt: connect to google drive

# from google.colab import drive
# drive.mount('/content/drive')

from math import radians, cos, sin, atan2, sqrt, degrees

In [19]:
import ee
import os
import math
from tqdm import tqdm

In [20]:
ee.Authenticate()
service_account = "lums-forest-fire-observatory@forest-fire-obs.iam.gserviceaccount.com"
credentials = ee.ServiceAccountCredentials(service_account, "forest-fire-obs-5966ba29fefa.json")
# credentials = ee.ServiceAccountCredentials(service_account, "/content/drive/MyDrive/google_earth_engine/forest-fire-obs-5966ba29fefa.json")
ee.Initialize(credentials=credentials, project='forest-fire-obs')
print(ee.String('Hello, Earth Engine!').getInfo())

Hello, Earth Engine!


In [21]:
def region_parameter(point, parameter,size):
  # Initialize an empty matrix with zeros
  matrix = [[0 for _ in range(size)] for _ in range(size)]

  # x = ((math.floor(size/2))- size)*1000+1
  # y = (size -(math.floor(size/2)))*1000+1

  x=(size*-1000) +1
  y=(size*1000) +1

  # Fill the matrix with numbers according to the specified conditions
  for i in range(size):
    y = y -1000
    # x = ((math.floor(size/2))- size)*1000+1
    x=(size*-1000) +1
    for j in range(size):
      x = x+ 1000
      location = point.buffer(y,x).centroid()
      print('x= ', x , 'y= ',y)

      # Load the image collection
      image = ee.ImageCollection("NOAA/GFS0P25").filterDate('2024-01-01', '2024-01-02').first()
      # Get the parameter value at the current point
      parameter_value = image.reduceRegion(ee.Reducer.first(), location).get(parameter).getInfo()

      # Add the parameter value to the matrix
      matrix[i][j] = parameter_value

  return matrix


In [22]:
# Define the original point of interest (Lahore, Pakistan)
original_point = ee.Geometry.Point(74.3587, 31.5204)
matrix_size = 3
# Get information for the square region around the original point for the parameter 'temperature_2m_above_ground'
region_info = region_parameter(original_point, 'temperature_2m_above_ground',matrix_size)

# Print the parameter values as a matrix
print("Parameter values for the square region around the original point:")
for row in region_info:
    print(row)

x=  -1999 y=  2001
x=  -999 y=  2001
x=  1 y=  2001
x=  -1999 y=  1001
x=  -999 y=  1001
x=  1 y=  1001
x=  -1999 y=  1
x=  -999 y=  1
x=  1 y=  1
Parameter values for the square region around the original point:
[7.824853515625023, 7.824853515625023, 7.824853515625023]
[7.824853515625023, 7.824853515625023, 7.824853515625023]
[7.824853515625023, 7.824853515625023, 7.824853515625023]


In [24]:
import ee
import math

def coordinates_1km_right(lat, lng):
    earth_radius = 6371.0  # Radius of the Earth in km

    lat_rad = radians(lat)
    lng_rad = radians(lng)

    angular_distance = 1/earth_radius

    new_lat_rad = lat_rad
    new_lng_rad = lng_rad + atan2(sin(angular_distance) * cos(lat_rad), cos(angular_distance) - sin(lat_rad) * sin(lat_rad))

    new_lat = degrees(new_lat_rad)
    new_lng = degrees(new_lng_rad)

    return new_lat, new_lng

def coordinates_1km_below(lat, lng):
    earth_radius = 6371.0  # Radius of the Earth in km

    lat_rad = radians(lat)
    lng_rad = radians(lng)

    angular_distance = 1/earth_radius

    new_lat_rad = lat_rad - angular_distance

    new_lon_rad = lng_rad

    new_lat = degrees(new_lat_rad)
    new_lng = degrees(new_lon_rad)

    return new_lat, new_lng

def region_parameter(longitude, latitude, size):
    # Initialize an empty dictionary to store parameter values
    location = ee.Geometry.Point(longitude, latitude)
    parameters = {
        'wind_direction': [],
        'wind_speed': [],
        'min_temperature': [],
        'max_temperature': [],
        'humidity': [],
        'precipitation': [],
        'vegetation': [],
        'elevation': [],
        'population_density': [],
        'drought_index': [],
        'energy_release_component': [],
        'coordinates': []
    }

    # x = ((math.floor(size / 2)) - size) * 1000 + 1
    # y = (size - (math.floor(size / 2))) * 1000 + 1

    # x=(size*-1000) +1
    # y=(size*1000) +1
    original_longitude = longitude
    original_latitude = latitude
    # Loop over the grid
    for i in range(size):
        # y = y - 1000
        # x = ((math.floor(size / 2)) - size) * 1000 + 1
        # x=(size*-1000) +1


        for j in tqdm(range(size)):
            # x = x + 1000
            # location = point.buffer((x, y)).centroid()

            # # Get elevation
            elevation = ee.Image("CGIAR/SRTM90_V4").select('elevation')
            elevation_value = elevation.reduceRegion(ee.Reducer.first(), location).get('elevation').getInfo()

            # # Get wind direction
            wind_direction = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('2020-04-06', '2024-05-06').select('th').first()
            wind_direction_value = wind_direction.reduceRegion(ee.Reducer.first(), location).get('th').getInfo()

            # Get wind speed
            wind_speed = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1979-01-01', '2024-05-06').select('vs').first()
            wind_speed_value = wind_speed.reduceRegion(ee.Reducer.first(), location).get('vs').getInfo()

            # Get min temperature
            min_temperature = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1979-01-01', '2024-05-06').select('tmmn').first()
            min_temperature_value = min_temperature.reduceRegion(ee.Reducer.first(), location).get('tmmn').getInfo()

            # Get max temperature
            max_temperature = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1979-01-01', '2024-05-06').select('tmmx').first()
            max_temperature_value = max_temperature.reduceRegion(ee.Reducer.first(), location).get('tmmx').getInfo()

            # # Get humidity
            humidity = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1979-01-01', '2024-05-06').select('sph').first()
            humidity_value = humidity.reduceRegion(ee.Reducer.first(), location).get('sph').getInfo()

            # Get precipitation
            precipitation = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1979-01-01', '2024-05-06').select('pr').first()
            precipitation_value = precipitation.reduceRegion(ee.Reducer.first(), location).get('pr').getInfo()

            # Get vegetation
            vegetation = ee.ImageCollection("NOAA/VIIRS/001/VNP13A1").filterDate('2012-01-17', '2024-04-14').select('NDVI').first()
            vegetation_value = vegetation.reduceRegion(ee.Reducer.first(), location).get('NDVI').getInfo()

             # Get population count
            population_count = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").filterDate('2000-01-01', '2020-01-01').first()
            population_count_value = population_count.reduceRegion(ee.Reducer.first(), location).get('population_count').getInfo()

            # Get PDSI
            pdsi = ee.ImageCollection("GRIDMET/DROUGHT").filterDate('1980-01-05', '2024-04-29').select('pdsi').first()
            pdsi_value = pdsi.reduceRegion(ee.Reducer.first(), location).get('pdsi').getInfo()

            # Get ERC
            erc = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1979-01-01', '2024-05-06').select('erc').first()
            erc_value = erc.reduceRegion(ee.Reducer.first(), location).get('erc').getInfo()

            # # Load the image collection
            # collection = ee.ImageCollection("NOAA/GFS0P25")
            # # Filter the collection by date
            # filtered_collection = collection.filterDate('2024-01-01', '2024-01-02')
            # # Select the parameter band
            # parameter_band = parameter
            # # Get the first image in the filtered collection
            # image = filtered_collection.first()
            # # Get the parameter value at the current point
            # parameter_value = image.reduceRegion(ee.Reducer.first(), location).get(parameter_band).getInfo()


            # Add values to the dictionary
            parameters['elevation'].append(elevation_value)
            parameters['wind_direction'].append(wind_direction_value)
            parameters['wind_speed'].append(wind_speed_value)
            parameters['min_temperature'].append(min_temperature_value)
            parameters['max_temperature'].append(max_temperature_value)
            parameters['humidity'].append(humidity_value)
            parameters['precipitation'].append(precipitation_value)
            parameters['vegetation'].append(vegetation_value)
            parameters['population_density'].append(population_count_value)
            parameters['drought_index'].append(pdsi_value)
            parameters['energy_release_component'].append(erc_value)
            parameters['coordinates'].append((longitude, latitude))

            latitude, longitude = coordinates_1km_right(latitude, longitude)
            location = ee.Geometry.Point(longitude, latitude)
        longitude = original_longitude
        latitude, longitude = coordinates_1km_below(latitude, longitude)
        location = ee.Geometry.Point(longitude, latitude)



    return parameters

# Define the original point of interest (Lahore, Pakistan)
longitude = -79.06736488554685
latitude = 35.760130294641336
original_point = ee.Geometry.Point(longitude, latitude)
matrix_size = 10  # Reduced matrix size for demonstration
# Get information for the square region around the original point
region_info = region_parameter(longitude, latitude, matrix_size)

# Print the parameter values and elevations as a dictionary
print("Parameter values and elevations for the square region around the original point:")
print(region_info)


100%|███████████████████████████████████████████| 10/10 [00:47<00:00,  4.79s/it]

Parameter values and elevations for the square region around the original point:
{'wind_direction': [274, 274, 274, 274, 274, 274, 274, 283, 283, 283, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278, 271, 271, 271, 271, 271, 271, 271, 278, 278, 278], 'wind_speed': [6.040984153747559, 6.040984153747559, 6.040984153747559, 6.1478590965271, 6.1478590965271, 6.1478590965271, 6.1478590965271, 6.256575584411621, 6.256575584411621, 6.256575584411621, 5.926466941833496, 5.926466941833496, 5.926466941833496, 6.022069931030273, 6.022069931030273, 6.022069931030273, 6.022069931030273, 6.119524002075195, 6.119524002075195, 6.119524002075195, 5.926466941